In [ ]:
import sympy as sp

sMul = sp.Mul

from symqups import s as CahillGlauberS
from symqups.objects.scalars import (q, p, alpha, alphaD, W, mu, _Primed, 
                                     _DerivativeSymbol, hbar, _deprime, StateFunction)
from symqups.objects.operators import qOp, pOp, annihilateOp, createOp, rho, Operator
from symqups.utils.algebra import define, qp2a, get_random_poly
from symqups.operations.operator_ordering import sOrdering, explicit, express
from symqups.operations.quantization import s_quantize, naive_quantize
from symqups.objects import scalars
from symqups._internal.cache import sub_cache
from symqups.operations.star_product import Bopp, Star, _replace_diff
from symqups.utils.multiprocessing import _mp_helper
from symqups._internal.basic_routines import operation_routine
from symqups._internal.operator_handling import (
    separate_operator,
    separate_term_by_polynomiality,
    separate_term_oper_by_sub,
    collect_alpha_type_oper_from_monomial_by_sub,
    get_oper_sub
)
from symqups.utils.multiprocessing import MP_CONFIG
from symqups._internal.warnings import suppress_warning_repeats
from symqups._internal.grouping import UnBoppable, NotAnOperator, PhaseSpaceObject

a = [alpha(i) for i in range(10)]
aop = [annihilateOp(i) for i in range(10)]
ad = [alphaD(i) for i in range(10)]
adop = [createOp(i) for i in range(10)]
x = sp.Symbol("x")
qq = [q(i) for i in range(10)]
pp = [p(i) for i in range(10)]
W = scalars.W
s.val = 1
scalars.mu = 1
scalars.hbar = 1

In [3]:
test = aop[2]**2*adop[1]*sp.exp(adop[1]*adop[0]) * aop[0]

test

\hat{a}^{\dagger}_{1}*exp(\hat{a}^{\dagger}_{0}*\hat{a}^{\dagger}_{1})*\hat{a}_{0}*\hat{a}_{2}**2

In [ ]:
class CGTransform(sp.Expr, UnBoppable, NotAnOperator):
    express = False
    
    def __new__(cls, expr : sp.Expr, s : sp.Number = None, *_internal):
        """
        oper -> quantum ps vars
        """
        
        if not(s):
            s = CahillGlauberS.val
        
        def treat_add(A : sp.Expr) -> sp.Expr:
            return sp.Add(*_mp_helper(A.args, CGTransform))
        
        def treat_substitutable(A : sp.Expr) -> sp.Expr:
            return A.subs({op(sub) : sc(sub) for sub in sub_cache for op,sc in [[annihilateOp, alpha],
                                                                                [createOp, alphaD]]})
        
        def treat_function(A : sp.Function) -> sp.Expr:
            sOrdering_of_A = sOrdering(A)
            if not(isinstance(sOrdering_of_A, sOrdering)):
                return treat_substitutable(sOrdering_of_A)
            return CGTransform(sOrdering_of_A)
        
        def treat_sOrdering(A : sOrdering, vars) -> sp.Expr:
            if cls.express and (A.args[1] != s.val):
                return CGTransform(A.express(s.val))
            
            if not(vars):
                vars = [sc(atom.sub) for sc,op in [[alpha, annihilateOp], 
                                                   [alphaD, createOp]] 
                        for atom in A.atoms(op)]
            
            return super(CGTransform, cls).__new__(cls, A, *vars)
        
        def treat_mul(A : sp.Expr) -> sp.Expr:
            return Star(*_mp_helper(A.args, CGTransform))
            
        expr = qp2a(sp.sympify(expr))
        return operation_routine(expr,
                                "CG_transform",
                                [],
                                [],
                                {Operator : expr},
                                {sp.Add : treat_add,
                                sp.Mul : treat_mul,
                                (Operator, sp.Pow) : treat_substitutable,
                                sp.Function : treat_function,
                                sOrdering : lambda A: treat_sOrdering(A, _internal)})
        
    def _latex(self, printer):
        notation = r"\mathcal{G}"
        if not(self.express):
            notation += r"_{s={%s}}" % sp.latex(s.val)
        return r"{%s}\left[{%s}\right]" % (notation, sp.latex(self.args[0]))

In [144]:
expr = CGTransform(sOrdering(aop[0]*adop[0]**2, 0.5) * sOrdering(aop[0]*adop[0], 0.5))
expr

Star(CGTransform(sOrdering(\hat{a}^{\dagger}_{0}**2*\hat{a}_{0}, 0.5), \alpha_{0}, \overline{\alpha}_{0}), CGTransform(sOrdering(\hat{a}^{\dagger}_{0}*\hat{a}_{0}, 0.5), \alpha_{0}, \overline{\alpha}_{0}))